In [3]:
!pip install hub

  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
  Using cached jmespath-1.0.0-py3-none-any.whl (23 kB)


In [4]:
import hub
ds = hub.load("hub://activeloop/gtsrb-train")

Opening dataset in read-only mode as you don't have write permissions.
hub://activeloop/gtsrb-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/gtsrb-train


Dataset(path='hub://activeloop/gtsrb-train', read_only=True, tensors=['images', 'boxes', 'labels', 'shapes', 'colors'])